# データサイエンス100本ノック（構造化データ加工編） - R

## はじめに
- 初めに以下のセルを実行してください
- 必要なライブラリのインポートとCSVファイルからのデータ読み込みを行います
- 利用が想定されるライブラリは以下セルでインポートしています
- その他利用したいライブラリがあれば適宜インストールしてください（!マークに続けてOSコマンドを入力することで、任意のubuntu Linuxコマンドが入力可能）
- 名前、住所等はダミーデータであり、実在するものではありません

In [1]:
# require('RPostgreSQL')
require('tidyr')
require('dplyr')
require('stringr')
require('caret')
require('lubridate')
require('rsample')
require('unbalanced')

# host <- 'db'
# port <- Sys.getenv()["PG_PORT"]
# dbname <- Sys.getenv()["PG_DATABASE"]
# user <- Sys.getenv()["PG_USER"]
# password <- Sys.getenv()["PG_PASSWORD"]

# con <- dbConnect(PostgreSQL(), host=host, port=port, dbname=dbname, user=user, password=password)
# df_customer <- dbGetQuery(con,"SELECT * FROM customer")
# df_category <- dbGetQuery(con,"SELECT * FROM category")
# df_product <- dbGetQuery(con,"SELECT * FROM product")
# df_receipt <- dbGetQuery(con,"SELECT * FROM receipt")
# df_store <- dbGetQuery(con,"SELECT * FROM store")
# df_geocode <- dbGetQuery(con,"SELECT * FROM geocode")

Loading required package: tidyr
Warning message:
"package 'tidyr' was built under R version 3.6.3"Loading required package: dplyr
Warning message:
"package 'dplyr' was built under R version 3.6.3"
Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union

Loading required package: stringr
Warning message:
"package 'stringr' was built under R version 3.6.3"Loading required package: caret
Warning message:
"package 'caret' was built under R version 3.6.3"Loading required package: lattice
Loading required package: ggplot2
Warning message:
"package 'ggplot2' was built under R version 3.6.3"Loading required package: lubridate

Attaching package: 'lubridate'

The following object is masked from 'package:base':

    date

Loading required package: rsample
Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
"

In [2]:
df_customer <- read.csv("./data/customer.csv", encoding="UTF-8")
head(df_customer, n = 5)

customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
CS021313000114,"大野 あや子,1",女性,1981-04-29,37,259-1113,神奈川県伊勢原市粟窪**********,S14021,20150905,0-00000000-0,
CS037613000071,六角 雅彦,9,不明,1952-04-01,66,136-0076,東京都江東区南砂**********,S13037,20150414,0-00000000-0
CS031415000172,"宇多田 貴美子,1",女性,1976-10-04,42,151-0053,東京都渋谷区代々木**********,S13031,20150529,D-20100325-C,
CS028811000001,"堀井 かおり,1",女性,1933-03-27,86,245-0016,神奈川県横浜市泉区和泉町**********,S14028,20160115,0-00000000-0,
CS001215000145,田崎 美紀,1,女性,1995-03-29,24,144-0055,東京都大田区仲六郷**********,S13001,20170605,6-20090929-2


In [3]:
# df_category <- read.csv("./data/category.csv", encoding="UTF-8")
df_category <- read.csv("./data/category.csv", encoding="shift-jis") # エラーがでたので文字コードを変更
head(df_category, n = 5)

category_major_cd,category_major_name,category_medium_cd,category_medium_name,category_small_cd,category_small_name
4,惣菜,401,御飯類,40101,弁当類
4,惣菜,401,御飯類,40102,寿司類
4,惣菜,402,佃煮類,40201,魚介佃煮類
4,惣菜,402,佃煮類,40202,海草佃煮類
4,惣菜,402,佃煮類,40203,野菜佃煮類


In [4]:
df_product <- read.csv("./data/product.csv", encoding="UTF-8")
head(df_product, n = 5)

X.U.FEFF.product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost
P040101001,4,401,40101,198,149
P040101002,4,401,40101,218,164
P040101003,4,401,40101,230,173
P040101004,4,401,40101,248,186
P040101005,4,401,40101,268,201


In [5]:
df_receipt <- read.csv("./data/receipt.csv", encoding="UTF-8")
head(df_receipt, n = 5)

sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount
20181103,1257206400,S14006,112,1,CS006214000001,P070305012,1,158
20181118,1258502400,S13008,1132,2,CS008415000097,P070701017,1,81
20170712,1215820800,S14028,1102,1,CS028414000014,P060101005,1,170
20190205,1265328000,S14042,1132,1,ZZ000000000000,P050301001,1,25
20180821,1250812800,S14025,1102,2,CS025415000050,P060102007,1,90


In [6]:
df_store <- read.csv("./data/store.csv", encoding="shift-jis") # 正常に読まなかったので文字コードを変換した
head(df_store, n = 5)

store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
S12014,千草台店,12,千葉県,千葉県千葉市稲毛区千草台一丁目,チバケンチバシイナゲクチグサダイイッチョウメ,043-123-4003,140.1180,35.63559,1698
S13002,国分寺店,13,東京都,東京都国分寺市本多二丁目,トウキョウトコクブンジシホンダニチョウメ,042-123-4008,139.4802,35.70566,1735
S14010,菊名店,14,神奈川県,神奈川県横浜市港北区菊名一丁目,カナガワケンヨコハマシコウホククキクナイッチョウメ,045-123-4032,139.6326,35.50049,1732
S14033,阿久和店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4043,139.4961,35.45918,1495
S14036,相模原中央店,14,神奈川県,神奈川県相模原市中央二丁目,カナガワケンサガミハラシチュウオウニチョウメ,042-123-4045,139.3716,35.57327,1679


In [7]:
df_geocode <- read.csv("./data/geocode.csv", encoding="UTF-8") # 時間がかかる, Python では一瞬で読めた
head(df_geocode, n = 5)

X.U.FEFF.postal_cd,prefecture,city,town,street,address,full_address,longitude,latitude
060-0000,"北海道,札幌市中央区",,,,北海道札幌市中央区,141.34103,43.05513,NA
064-0941,"北海道,札幌市中央区","旭ケ丘,",,"北海道札幌市中央区旭ケ丘,141.31972",43.04223,,NA,NA
060-0042,"北海道,札幌市中央区",大通西,,１丁目,北海道札幌市中央区大通西１丁目,141.35637,43.06102,NA
060-0042,"北海道,札幌市中央区",大通西,,２丁目,北海道札幌市中央区大通西２丁目,141.35445,43.06080,NA
060-0042,"北海道,札幌市中央区",大通西,,３丁目,北海道札幌市中央区大通西３丁目,141.35275,43.06086,NA


# 演習問題

---
> R-026: レシート明細データフレーム（df_receipt）に対し、顧客ID（customer_id）ごとに最も新しい売上日（sales_ymd）と古い売上日を求め、両者が異なるデータを10件表示せよ。

In [8]:
head(df_receipt)

sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount
20181103,1257206400,S14006,112,1,CS006214000001,P070305012,1,158
20181118,1258502400,S13008,1132,2,CS008415000097,P070701017,1,81
20170712,1215820800,S14028,1102,1,CS028414000014,P060101005,1,170
20190205,1265328000,S14042,1132,1,ZZ000000000000,P050301001,1,25
20180821,1250812800,S14025,1102,2,CS025415000050,P060102007,1,90
20190605,1275696000,S13003,1112,1,CS003515000195,P050102002,1,138


In [9]:
dim(df_receipt)

[1] 104681      9

In [10]:
df_receipt %>%
    group_by(customer_id) %>%
    summarise(max_ymd=max(sales_ymd)
             ,min_ymd=min(sales_ymd)) %>%
    filter(max_ymd != min_ymd) %>%
    slice(1:10)

`summarise()` ungrouping output (override with `.groups` argument)


customer_id,max_ymd,min_ymd
CS001114000005,20190731,20180503
CS001115000010,20190405,20171228
CS001205000004,20190625,20170914
CS001205000006,20190224,20180207
CS001214000009,20190902,20170306
CS001214000017,20191006,20180828
CS001214000048,20190929,20171109
CS001214000052,20190617,20180208
CS001215000005,20181021,20170206
CS001215000040,20171022,20170214


In [11]:
# 上記が公式解答、順次分解して理解
# group_by(): dplyrの関数。ある変数の値でデータセットをグループ化します。これを行うことで、グループ単位で処理を行えるようになる
# https://kazutan.github.io/kazutanR/hands_on_170730/group_by.html
# summarise()：dplyrの関数。ここでは、sales_ymd の最小値と最大値を抽出
# https://qiita.com/yutannihilation/items/24d83ef83cdaad804555

In [12]:
df_tmp <- df_receipt %>% group_by(customer_id)
df_tmp2 <- df_tmp %>% summarise(max_ymd=max(sales_ymd) ,min_ymd=min(sales_ymd))
head(df_tmp2)

`summarise()` ungrouping output (override with `.groups` argument)


customer_id,max_ymd,min_ymd
CS001113000004,20190308,20190308
CS001114000005,20190731,20180503
CS001115000010,20190405,20171228
CS001205000004,20190625,20170914
CS001205000006,20190224,20180207
CS001211000025,20190322,20190322


In [13]:
dim(df_tmp2) # 104681行から8307行に絞りこまれている

[1] 8307    3

In [14]:
df_tmp3 <- df_tmp2 %>%  filter(max_ymd != min_ymd) # 最も新しい売上日と古い売上日が異なるものだけ抽出
head(df_tmp3)

customer_id,max_ymd,min_ymd
CS001114000005,20190731,20180503
CS001115000010,20190405,20171228
CS001205000004,20190625,20170914
CS001205000006,20190224,20180207
CS001214000009,20190902,20170306
CS001214000017,20191006,20180828


In [15]:
dim(df_tmp3) # 8307 から更に 5546 に絞り込まれた

[1] 5546    3

---
> R-027: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の平均を計算し、降順でTOP5を表示せよ。

In [16]:
df_receipt %>%
    group_by(store_cd) %>%
    summarise(mean_amount=mean(amount)) %>%
    arrange(desc(mean_amount)) %>%
    slice(1:5)

`summarise()` ungrouping output (override with `.groups` argument)


store_cd,mean_amount
S13052,402.8675
S13015,351.1120
S13003,350.9155
S14010,348.7913
S13001,348.4704


In [17]:
# summarise() で売上金額（amount）の平均を計算
df_tmp <- df_receipt %>% group_by(store_cd) %>% summarise(mean_amount=mean(amount))
head(df_tmp)

`summarise()` ungrouping output (override with `.groups` argument)


store_cd,mean_amount
S12007,307.6883
S12013,330.1941
S12014,310.8303
S12029,315.6239
S12030,288.5337
S13001,348.4704


In [18]:
dim(df_tmp) # 店舗数のみの平均、つまり52店舗しかない

[1] 52  2

In [19]:
# arrange(): dplyrの関数。順番を変える。降順なら arrange(desc(変数))とする
# https://qiita.com/matsuou1/items/e995da273e3108e2338e
df_tmp2 <- df_tmp %>% arrange(desc(mean_amount))
head(df_tmp2)

store_cd,mean_amount
S13052,402.8675
S13015,351.1120
S13003,350.9155
S14010,348.7913
S13001,348.4704
S13020,337.8799


In [20]:
# 昇順なら arrange(変数)
df_tmp3 <- df_tmp %>% arrange(mean_amount)
head(df_tmp3)

store_cd,mean_amount
S14042,278.3389
S12030,288.5337
S14050,290.5913
S14049,297.8168
S13037,299.9078
S14012,303.0278


In [21]:
# 店舗数を数える演習
# unique(): ある列に登場するユニークな（重複のない）値を返す。https://qiita.com/ymmy02/items/368c280f30bd5cd8d179
# length(): https://qiita.com/ymmy02/items/368c280f30bd5cd8d179
length(unique(df_receipt$store_cd))

[1] 52

---
> R-028: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の中央値を計算し、降順でTOP5を表示せよ。

In [22]:
# 27, 28番と同じパターン。中央値は summarise(median_amount=median(変数))
df_receipt %>%
    group_by(store_cd) %>%
    summarise(median_amount=median(amount)) %>%
    arrange(desc(median_amount)) %>%
    slice(1:5)

`summarise()` ungrouping output (override with `.groups` argument)


store_cd,median_amount
S13052,190
S14010,188
S14050,185
S13003,180
S13018,180


---
> R-029: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに商品コード（product_cd）の最頻値を求めよ。

In [23]:
table_product <-table(c(df_receipt$store_cd), df_receipt$product_cd)

store <- names(table_product[,1])

mode_product <- c()
for (i in 1:length(store)){
    mode_product[i] <- names(which.max(table_product[i,]))
}

data.frame(store_cd = store, store_cd = mode_product)

store_cd,store_cd.1
1,P060303001
2,P060303001
3,P060303001
4,P060303001
5,P060303001
6,P060303001
7,P060303001
8,P071401001
9,P060303001
10,P040503001


In [24]:
# 一行ずつ解読
# まず store_cdのベクトルを作り、product_cd とのテーブル作成
table_product <-table(c(df_receipt$store_cd), df_receipt$product_cd)
head(table_product)

   
    P040101001 P040101002 P040101003 P040101004 P040101005 P040101006
  1         11          5          3          0          3          0
  2          3          4          1          0          0          4
  3         16          1          6          0          1          0
  4         10         16          2          1          6          2
  5          7          3          5          1          6          6
  6         12          5          8          7          5          4
   
    P040101007 P040101008 P040101009 P040101010 P040102001 P040102002
  1          0          1          3          3         14          4
  2          0          2          5          4          9          6
  3          0          4          0          0          0          1
  4          1          3          1          3         12          1
  5          1          0          4          0          0          0
  6          2          3          0          2         17          2
   
    P040

In [30]:
store <- names(table_product[,1])
store

[1] "1"  "2"  "3"  "4"  "5"  "6"  "7"  "8"  "9"  "10" "11" "12" "13" "14" "15"
[16] "16" "17" "18" "19" "20" "21" "22" "23" "24" "25" "26" "27" "28" "29" "30"
[31] "31" "32" "33" "34" "35" "36" "37" "38" "39" "40" "41" "42" "43" "44" "45"
[46] "46" "47" "48" "49" "50" "51" "52"

In [33]:
mode_product <- c() # 空のベクトル

# storeの数(52)回ループを回して、mode_productの最頻値ををベクトル代入
# ベクトルの最大・最小要素の位置を求める関数として which.max(), which.min() が用意されている
# http://cse.naro.affrc.go.jp/takezawa/r-tips/r/13.html
for (i in 1:length(store)){ 
    mode_product[i] <- names(which.max(table_product[i,]))
}
mode_product

[1] "P060303001" "P060303001" "P060303001" "P060303001" "P060303001"
 [6] "P060303001" "P060303001" "P071401001" "P060303001" "P040503001"
[11] "P060303001" "P060303001" "P071401001" "P071102001" "P060101002"
[16] "P071401001" "P071401001" "P071401001" "P060303001" "P060303001"
[21] "P040503001" "P060303001" "P060303001" "P071401001" "P071401001"
[26] "P060303001" "P060303001" "P050102001" "P050101001" "P060303001"
[31] "P060303001" "P060101001" "P060303001" "P060101001" "P060303001"
[36] "P071401001" "P060303001" "P060303001" "P071401001" "P060303001"
[41] "P060303001" "P071401001" "P060303001" "P040503001" "P060303001"
[46] "P050101001" "P060303001" "P060303001" "P060303001" "P050101001"
[51] "P060303001" "P060303001"

In [34]:
# データフレームの作成
data.frame(store_cd = store, store_cd = mode_product)

store_cd,store_cd.1
1,P060303001
2,P060303001
3,P060303001
4,P060303001
5,P060303001
6,P060303001
7,P060303001
8,P071401001
9,P060303001
10,P040503001


---
> R-030: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の標本分散を計算し、降順でTOP5を表示せよ。

In [35]:
# 標本分散を計算する関数を定義
# http://strawberry-kyon.hateblo.jp/entry/2017/09/14/115631
var_sample <- function(x){ var(x)*(length(x)-1)/length(x) }

df_receipt %>%
    group_by(store_cd) %>%
    summarise(var_amount=var_sample(amount)) %>%
    arrange(desc(var_amount)) %>%
    slice(1:5)

`summarise()` ungrouping output (override with `.groups` argument)


store_cd,var_amount
S13052,440088.7
S14011,306314.6
S14034,296920.1
S13001,295432.0
S13015,295294.4
